In [8]:
import json 
from pathlib import Path
import shutil
import datetime as dt
from pprint import pprint

import pandas as pd

from wattile.entry_point import init_logging
from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import prep_for_rnn
from wattile.models import ModelFactory


PROJECT_DIRECTORY = Path().resolve().parents[1]


In [2]:
"""
This demo runs predictions on a pre-trained model in ./notebooks/exp_dir.
To train a model in that directory, see the notebook titled "training".
"""
exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"

if not exp_dir.exists():
    print("no trained model")
    exit()

with open(exp_dir / "configs.json", "r") as f:
    configs = json.load(f)

prontconfigs

{'data_input': {'data_dir': '/Users/heslinge/Repos/Wattile/tests/data/Synthetic Site',
  'data_config': 'Synthetic Site Config.json',
  'start_time': '2018-01-01T00:00:00-07:00',
  'end_time': '2022-01-01T00:00:00-07:00',
  'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
   'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'Synthetic Weather Station Direct Normal Irradiance',
   'Synthetic Weather Station Dry Bulb Temperature',
   'Synthetic Weather Station Global Horizontal Irradiance',
   'Synthetic Weather Station Relative Humidity',
   'Synthetic Weather Station Wind Speed'],
  'target_var': 'Synthetic Site Electricity Main Total Power'},
 'data_output': {'exp_dir': '/Users/heslinge/Repos/Wattile/notebooks/exp_dir',
  'plot_comparison': True,
  'plot_comparison_portion_start': 0.0,
  'plot_comparison_portion_end': 1.0},
 'data_processing': {'feat_time': {'month_of_year': ['sincos'],
   'day_of_week': ['binary_reg', 'binary_fuzzy'],
   'hour_of_d

In [3]:
"""
Init the model with it's configs. Be sure to switch it's use case to prediction.
"""

init_logging(local_results_dir=configs["data_output"]["exp_dir"])
model = ModelFactory.create_model(configs)
model.configs["learning_algorithm"]["use_case"] = "prediction"

model

Logging to: /Users/heslinge/Repos/Wattile/notebooks/exp_dir/output.out, PID: 99708


In [4]:
"""
If we want to get predictions for only one row, we have to find the time range needed to predict the nomial time.

we can use `get_input_window_for_output_time` to fgure this out.
"""
nominal_time = pd.Timestamp(year=2020, month=1, day=1, tz=dt.timezone.utc)

start_time, end_time = model.get_input_window_for_output_time(nominal_time)
freq = model.configs["data_processing"]["resample"]["bin_interval"]
inclusive = model.configs["data_processing"]["resample"]["bin_label"]

predictors = model.configs["data_input"]["predictor_columns"]
target_var = model.configs["data_input"]["target_var"]

print(f"""
    To use this model to predict {target_var} for {nominal_time}, we will need to feed it data:
        - starting at {start_time}
        - ending at {end_time}
        - a rough frequency of {freq}
        - predictors named {predictors}
""")


    To use this model to predict Synthetic Site Electricity Main Total Power for 2020-01-01 00:00:00+00:00, we will need to feed it data:
        - starting at 2019-12-31 18:00:00+00:00
        - ending at 2020-01-01 00:15:00+00:00
        - a rough frequency of 15min
        - predictors named ['Synthetic Weather Station Dew Point Temperature', 'Synthetic Weather Station Diffuse Horizontal Irradiance', 'Synthetic Weather Station Direct Normal Irradiance', 'Synthetic Weather Station Dry Bulb Temperature', 'Synthetic Weather Station Global Horizontal Irradiance', 'Synthetic Weather Station Relative Humidity', 'Synthetic Weather Station Wind Speed']



In [5]:
"""
Great, now lets make some dummy data that matchs those specs.
"""

data = pd.DataFrame(index=pd.date_range(start_time, end_time, freq=freq, inclusive=inclusive))
for predictor in predictors:
    data[predictor] = data.index.hour * 100 + data.index.minute

# even though we are predicting, we must fill the target column with dummy data. It's not used though.
data[target_var] = data.index.hour * 100 + data.index.minute


data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
2019-12-31 18:00:00+00:00,1800,1800,1800,1800,1800,1800,1800,1800
2019-12-31 18:15:00+00:00,1815,1815,1815,1815,1815,1815,1815,1815
2019-12-31 18:30:00+00:00,1830,1830,1830,1830,1830,1830,1830,1830
2019-12-31 18:45:00+00:00,1845,1845,1845,1845,1845,1845,1845,1845
2019-12-31 19:00:00+00:00,1900,1900,1900,1900,1900,1900,1900,1900
2019-12-31 19:15:00+00:00,1915,1915,1915,1915,1915,1915,1915,1915
2019-12-31 19:30:00+00:00,1930,1930,1930,1930,1930,1930,1930,1930
2019-12-31 19:45:00+00:00,1945,1945,1945,1945,1945,1945,1945,1945
2019-12-31 20:00:00+00:00,2000,2000,2000,2000,2000,2000,2000,2000
2019-12-31 20:15:00+00:00,2015,2015,2015,2015,2015,2015,2015,2015


In [6]:
"""
Let prep those for the model.
"""
train_df, val_df = prep_for_rnn(configs, data)
val_df

DatetimeIndex(['2019-12-31 18:00:00+00:00', '2019-12-31 18:15:00+00:00',
               '2019-12-31 18:30:00+00:00', '2019-12-31 18:45:00+00:00',
               '2019-12-31 19:00:00+00:00', '2019-12-31 19:15:00+00:00',
               '2019-12-31 19:30:00+00:00', '2019-12-31 19:45:00+00:00',
               '2019-12-31 20:00:00+00:00', '2019-12-31 20:15:00+00:00',
               '2019-12-31 20:30:00+00:00', '2019-12-31 20:45:00+00:00',
               '2019-12-31 21:00:00+00:00', '2019-12-31 21:15:00+00:00',
               '2019-12-31 21:30:00+00:00', '2019-12-31 21:45:00+00:00',
               '2019-12-31 22:00:00+00:00', '2019-12-31 22:15:00+00:00',
               '2019-12-31 22:30:00+00:00', '2019-12-31 22:45:00+00:00',
               '2019-12-31 23:00:00+00:00', '2019-12-31 23:15:00+00:00',
               '2019-12-31 23:30:00+00:00', '2019-12-31 23:45:00+00:00',
               '2020-01-01 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='15T')
DatetimeIndex(['2019-12-

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,HOD_binary_reg_0_min_lag24,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
2020-01-01 00:00:00+00:00,1800.0,1800.0,1800.0,1800.0,1800.0,1800.0,1800.0,-1.0,-1.836970e-16,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.017213,0.999852,0


In [7]:
"""
One line in, one prediction (with many quantiles) out.
"""
results = model.predict(val_df)

results

<xarray.DataArray (timestamp: 1, quantile: 9, horizon: 1)>
array([[[100.85417 ],
        [103.63314 ],
        [105.088715],
        [106.57381 ],
        [107.18616 ],
        [108.4029  ],
        [109.58919 ],
        [110.38441 ],
        [111.67562 ]]], dtype=float32)
Coordinates:
  * timestamp  (timestamp) object 2020-01-01T00:00:00+00:00
  * quantile   (quantile) float64 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9
  * horizon    (horizon) timedelta64[ns] 00:00:00